# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# review different rows in the establishments collection
pprint(db.establishments.find_one())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}

# Use count_documents to display the number of documents in the result
print("The number of documents with a hygiene score of 20 is:", establishments.count_documents(query))

# Display the first document in the results using pprint
pprint(db.establishments.find_one(query))

In [ ]:
collection_results = list(establishments.find(query))

In [ ]:
# Convert the result to a Pandas DataFrame
pandas_df = pd.DataFrame(collection_results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(pandas_df))

# Display the first 10 rows of the DataFrame
pandas_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query_rating = {'LocalAuthorityName': {'$regex': 'London'}, 
         'RatingValue': {'$gte': '4'}}

# Use count_documents to display the number of documents in the result
print("Number of establishments in London with a RatingValue greater than or equal to 4 is:", establishments.count_documents(query_rating))


# Display the first document in the results using pprint
pprint(db.establishments.find_one(query_rating))


In [ ]:
# Save the results to a variable first.
rating_results = list(establishments.find(query_rating))

# Convert the result to a Pandas DataFrame
query_rating_df = pd.DataFrame(rating_results)

# Display the number of rows in the DataFrame
print("Number of Rows in the DataFrame: ", len(query_rating_df))

# Display the first 10 rows of the DataFrame
query_rating_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

latitude_val = establishments.update_many({}, [
                                                {'$set':
                                                        {"geocode.latitude":
                                                            {'$toDecimal': "$geocode.latitude"}
                                                        }
                                                }
                                            ]
                                      )

longitud_val = establishments.update_many({}, [
                                                {'$set':
                                                        {"geocode.longitude":
                                                            {'$toDecimal': "$geocode.longitude"}
                                                        }
                                                }
                                            ]
                                         )

# Set the limit to 5
limit = 5
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query_geolocation = {'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search}, 
         'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
         'RatingValue': '4'
        }
sort =  [('scores.Hygiene', -1)]


# Print the results
pprint(list(establishments.find(query_geolocation).sort(sort).limit(limit)))



In [ ]:
# Save the results to a variable after running the query as a list.
results_final = list(establishments.find(query_geolocation).sort(sort).limit(limit))

# Convert result to Pandas DataFrame
query_df = pd.DataFrame(results_final)

# Print number of rows in the DataFrame
print("Number of Rows in the DataFrame: ", len(query_df))

# Display the top 10 rows of the DataFrame
query_df.head(10)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_establishments = {'$match': {'scores.Hygiene': {'$eq': 0}}}

# 2. Groups the matches by Local Authority
group_by_localAuthority = {'$group': {'_id': "$LocalAuthorityName", 'count': { '$sum': 1 }}}

# 3. Sorts the matches from highest to lowest
sort_matches = {'$sort': { 'count': -1 }}

# Build the pipeline
pipeline = [match_establishments, group_by_localAuthority, sort_matches]

# Run the pipeline through the aggregate method, cast the results as a list, and save the results to a variable
pipeline_results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print("Total number of establishments in each Local Authority area that have a hygiene score of 0: ", len(pipeline_results))

# Print the first 10 results
pprint(pipeline_results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
pipeline_df = pd.DataFrame(pipeline_results)

# Display the number of rows in the DataFrame
print("Number of Rows in DataFrame: ", len(pipeline_df))

# Display the first 10 rows of the DataFrame
pipeline_df.head(10)